# Setting up Kubernetes Endpoints
***
Ben Galewsky, Sr. Research Programmer

National Center for Supercomputing Applications

University of Illinois Urbana Champaign



# What is Kubernetes?
> Kubernetes, also known as K8s, is an open-source system for automating deployment, scaling, and management of <font color="red">*containerized*</font> applications.
- Deploy Docker Containers
- Automatically scale up and down according to need



# How are Applications Deployed?
We use [Helm](https://helm.sh), a package manager for helm applications.

- Declarative description of the application
- Uses a templating language with user provided configuaration values
- Packages are served out of public repositories


# FuncX Kubernetes Endpoint
[Helm Chart](https://github.com/funcx-faas/funcX/blob/main/helm/README.md) takes a handful of values. Most of the defaults can be used in the majority of deployments. Some key values to provide:
- image.tag
- workerImage
- minBlocks
- maxBlocks
- endpointUUID

# Authentication
Your funcX tokens are securly mounted in the cluster as a Kubernetes Secret.

Full instructions in the [Endpoint Helm Chart README](https://github.com/funcx-faas/funcX/blob/main/helm/README.md#how-to-use)

# Today's Demonstration
Use case: want to be able to use the GDAL Geospatial Library to compute GIS Stuff. The python GDAL library is not easy to install since it depends on C++ libraries.

1. Create a Dockerfile that will run python 3.7 GDAL libraries
2. Create a values.yaml file for our specific endpoint
3. Deploy the funcx_endpoint helm chart with these values
4. Define a function that computes the area of a geomoetry
5. Call it!

# Dockerfile
Based on [thinkWhere Docker Image](https://github.com/thinkWhere/GDAL-Docker) - needed to freshen up the version of Ubuntu it is based on.

In [ ]:
FROM python:3.7-buster
RUN apt-get update
RUN apt-get upgrade -y

# Install GDAL dependencies
RUN apt-get install -y python3-pip libgdal-dev locales

# Ensure locales configured correctly
RUN locale-gen en_US.UTF-8
ENV LC_ALL='en_US.utf8'

# Update C env vars so compiler can find gdal
ENV CPLUS_INCLUDE_PATH=/usr/include/gdal
ENV C_INCLUDE_PATH=/usr/include/gdal

# This will install latest version of GDAL
RUN pip3 install GDAL==2.4.2


# Values.yaml
This is the configuration used to control the endpoint that is deployed by the helm chart. 

Specify the docker image to use in the worker, as well as the number of workers that can be scaled up to.


In [ ]:

image:
  tag: main-3.7  # The endpoint must use the same python version as the worker and the client.


# Image to use in the worker
workerImage: bengal1/gdal-python:3.7-ubuntu
        
# This command is run on each worker upon startup. Install the current version of funcX endpoint
workerInit: python3 -m pip install funcx-endpoint>=0.3.0
    
# How many workers do we start with and what is the maximum we can scale up to?
minBlocks: 1
maxBlocks: 2

# If we want to keep a UUID for this endpoint from an earlier deployment set it here.
endpointUUID:


detachEndpoint: true
workerNamespace: default

# Live Demo
Deploying a Kubernetes endpoint using helm chart

# Exercise the Endpoint
The following cells will connect to FuncX and register a function that computes the area of a GDAL polygon

In [55]:
from funcx.sdk.client import FuncXClient, TaskPending

fxc = FuncXClient()

gdal_endpoint = '' # Paste the endpoint UUID from the deployment 


In [56]:
def compute_area(wkt1):
    from osgeo import ogr
    poly = ogr.CreateGeometryFromWkt(wkt1)

    return poly.GetArea()

area_uuid = fxc.register_function(compute_area)

In [ ]:
task_id = fxc.run(endpoint_id=gdal_endpoint, function_id=area_uuid, 
                  wkt1="POLYGON ((1208064.271243039 624154.6783778917, 1208064.271243039 601260.9785661874, 1231345.9998651114 601260.9785661874, 1231345.9998651114 624154.6783778917, 1208064.271243039 624154.6783778917))")
task_id



In [ ]:
try:
    print(fxc.get_result(task_id))
except TaskPending as pending:
    print(pending)